# Mini projet Qualité de Données : Détections des doublons
## ***Christophe COMPAIN / Sander COHEN***

### Objectif et Méthodologie suivie
L'objectif du projet est d'identifier les logiciels vendus sur les deux plateformes.
Pour ce faire, nous disposons des données pour chacune des plateformes isolément, respectivement dans les fichiers ***Company1.csv*** et ***Company2.csv***.

Au cours de l'étude, nous avons essayé nombre de prétraitements et algorithmes de recherche de doublons différents. Nous ne présenterons ici que ceux finalement sélectionnés, et ne ferons qu'évoquer une partie de ceux mis de côté (ceux ayant nous semblant le plus interessant dans l'idée même s'il ne permettent pas d'améliorer significativement les résultats).

Après imports des packages et données, nous explorerons brièvement dans une première partie les données et en tirerons des points d'attention pour le prétraitement et la détection des doublons.
Dans une deuxième partie, nous présenterons et effectuerons notre pré-traitement. La troisième partie comportera nos fonctions et algorithmes de detections de doublons, ainsi que nos résultats.


### 0 - Import des packages utilisés et import des données

In [1]:
import pandas as pd
import nltk
import time
import numpy as np
import math
import re
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\scohe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\scohe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
path = "D:\\OneDrive - Université Paris-Dauphine\\Bureau\\Cours Master\\12-Qualité de Données\\\Projet\\mini-projet\\"
file1= "Data\\Company1.csv" #"SampleData\\Sample_Company1.csv"
file2= "Data\\Company2.csv" #"SampleData\\Sample_Company2.csv"
real= "Data\\Ground_truth_mappings.csv" #"SampleData\\Sample_Groud_truth_mappings.csv"
#path = "/Users/ccompain/Documents/code/Dauphine/MasterIASD_prepa_data/mini-projet/github/"
#file1 = "Data/Company1.csv"
#file2 = "Data/Company2.csv"
#real= "Data/Ground_truth_mappings.csv" #"SampleData\\Sample_Groud_truth_mappings.csv"

In [3]:
company1 = pd.read_csv(path+file1, encoding = "ISO-8859-1")
company2 = pd.read_csv(path+file2, encoding = "ISO-8859-1")
ground_truth_matches = pd.read_csv(path+real, encoding = "ISO-8859-1").drop_duplicates()

### 1 - Exploration des données

#### Aspects généraux

In [4]:
company1.head(5)

,id,title,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99


In [5]:
company2.head(5)

,id,name,description,manufacturer,price
0,11125907881740407428,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99
1,11538923464407758599,superstart! fun with reading & writing!,fun with reading & writing! is designed to hel...,NaN,8.49
2,11343515411965421256,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for re...,intuit,637.99
3,12049235575237146821,math missions: the amazing arcade adventure (g...,save spectacle city by disrupting randall unde...,NaN,12.95
4,12244614697089679523,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from ...,adobe software,805.99


Nous pouvons noter que :
* les produits semblent être des logiciels informatiques, et les 'manufacturer' des éditeurs;
* les noms des différentes colonnes correspondent entre les deux dataframes, à l'exception de la colonne 'title' de ***company1*** qui correspond à 'name' de ***company2***;
* dans ***company 1***, plusieurs 'description' sont en 'NaN' et plusieurs 'price' sont à 0;
* dans ***company 2***, plusieurs 'manufacturer' sont en 'NaN'.

Une analyse plus poussée nous a permis également de detecter d'autres caractéristiques à retraiter :
* certains prix sont affichés en livres sterling dans ***company 2***. Ne connaissant pas la devise initiale, nous retraiterons uniquement la chaine de caractères en supprimant les caractères non numériques;
* certaines 'description' sont en 'NaN' également dans ***company 2***;
* les deux dataframes comportent des typo.

Afin de détecter l'ampleur des 'NaN', nous effectuons un *count* sur les deux dataframes :



In [6]:
company1.count()

id              1363
title           1363
description     1248
manufacturer    1363
price           1363
dtype: int64

In [7]:
company2.count()

id              3226
name            3226
description     3035
manufacturer     232
price           3226
dtype: int64

Nous observons que :
* 'manufacturer' est très peu rempli dans ***company2***;
* 'description' est rempli pour plus de 90% des deux dataframes;
* ***company2*** contient plus de 2.3 fois plus d'entrées que ***company1***. 

Concernant ce dernier point, cela peut laisser présager des doublons multiples (un logiciel présent dans ***company1*** pourrait avoir plusieurs matches dans ***company2***). Dans la pratique, c'est effectivement ce que nous observons dans le dataframe ***ground_truth_matches***.

#### Top manufacturers

In [8]:
company1.groupby('manufacturer')['id'].count().sort_values(ascending=False).head(9)

manufacturer
adobe                   87
encore software         76
topics entertainment    73
encore                  62
microsoft               58
aspyr media             27
apple                   26
fogware publishing      19
intuit                  18
Name: id, dtype: int64

In [9]:
company2.groupby('manufacturer')['id'].count().sort_values(ascending=False).head(9)

manufacturer
punch software         30
abacus software        27
freeverse software     24
individual software    24
apple software         15
intuit                  8
webroot software        7
onone software          7
ce software             6
Name: id, dtype: int64

Avant retraitement des 'NaN' de 'manufacturer' dans ***company2***, nous observons que:
* les 'manufacturer' principaux sont différents dans les deux dataframes;
* certains 'manufacturer' sont comptés deux fois comme 'encore' et 'encore software' dans ***company1***.

### 2  - Prétraitement des données

Dans cette partie, nous allons effectuer un prétraitement sur les dataframes ***company1*** et ***company2***, et notamment :
* créer une fonction *prep* qui contiendra notre prétraitement des colonnes string utilisées qui :
    1. supprimera les caractères non alphanumériques, 
    2. convertira les caractères en minuscules,
    3. remplacera certains mots choisis (exemples : 'professional' -> 'pro', 'windows' -> 'win'),
    4. après tokenization, supprimera des stopwords qui nous semblent polluer la détection de doublons (stopwords de nltk agrementé de quelques mots parmi les plus utilisés de la base comme 'software', 'entertainment', 'inc'...),
    5. lemmatisera les mots de façon à limiter les typo, pluriels...
    6. renverra la chaine de caractères comprenant l'ensemble des mots séparés par des espaces
* créer une fonction *retreatprice* qui permettra de conserver uniquement les caractères numériques (et notamment supprimer les 'gbp') et renvoit le résultat sous forme de 'float';
* normaliser les deux dataframes en :
    1. renommant la colonne 'title' de ***company 1*** en 'name';
    2. remplaçant les 'NaN' par des ' ' dans les différentes colonnes;
    3. appliquant la fonction *retreatprice* à 'price' pour obtenir une nouvelle colonne 'price_retreat'
    4. appliquant la fonction *prep* à la concatenation des colonnes 'manufacturer' et 'name' pour obtenir une nouvelle colonne 'full data'. Cette colonne sera à la base de notre calcul de similarité
* ajouter une colonne 'Version', basée sur la recherche des termes 'win', 'mac' et 'linux' au sein des colonnes 'name' et 'description' (en cas de présence multiple ou d'absence, 'None' est renvoyé).

Nous n'incluons pas les codes d'autres tentatives non conservées comme, par exemple, la suppression au sein des colonnes 'full data' dans chacun des deux dataframes des mots uniques (ie qu'on ne retrouve qu'une fois dans tout le dataframe). Ceux-ci, différenciant, aurait pu polluer nos calculs de similarités (comme le font les codes existant dans ***company 2*** du type '22070152'), mais semblent au final permettre de limiter les faux positifs. Nous avons fait le choix de ne pas sélectionner à la main les mots uniques améliorant nos résultats pour ne pas artificiellement gonfler notre F-measure (ie en connaissant à priori ***ground_truth_matches***).


#### Récupération d'une base de stop_words utilisée dans la fonction prep

In [10]:
#Récupération d'une base de stop_words utilisée dans la fonction prep
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words.update(["r","v","software","entertainment","inc","usa"])

#### Création de la fonction de nettoyage textuelle *prep*

In [11]:
def prep(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    texte = re.sub("[^a-zA-Z0-9_]", " ",str(texte)).lower()
    #remplacement de mots
    texte = texte.replace("professional", "pro").replace("windows","win").replace("upgrade","upg").replace("deluxe","dlx")
    #tokenization par mot
    tokens = nltk.word_tokenize(texte)
    #suppression des stopwords
    filtered_tokens = [w for w in tokens if not w in stop_words]
#    # Stemming
#    texte = [nltk.stem.SnowballStemmer('english').stem(w) for w in filtered_tokens]
    # Lemmatization
    texte = [nltk.stem.WordNetLemmatizer().lemmatize(w) for w in filtered_tokens]
    #remise sous forme d'une string
    return " ".join(texte)

#### Création de la fonction de normalisation des prix *retreatprice*


In [12]:
def retreatprice(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    return float(re.sub("[^0-9.]", " ",str(texte)))

#### Normalisation des dataframes

In [13]:
company1['Company']="company1"
company1=company1.rename(columns={"title": "name"})
company1['name'] = company1['name'].fillna(' ')
company1['manufacturer'] = company1['manufacturer'].fillna(' ')
company1['description'] = company1['description'].fillna(' ')
company1['price'] = company1['price'].fillna(' ')
company1['price_retreat'] = company1['price'].apply(retreatprice)
company1['full data']=company1['manufacturer'].apply(prep) + ' ' + company1['name'].apply(prep) # + ' ' + company1['description'].apply(prep)

company2['Company']="company2"
company2['name'] = company2['name'].fillna(' ')
company2['manufacturer'] = company2['manufacturer'].fillna(' ')
company2['description'] = company2['description'].fillna(' ')
company2['price'] = company2['price'].fillna(' ')
company2['price_retreat'] = company2['price'].apply(retreatprice)
company2['full data']=company2['manufacturer'].apply(prep) + ' ' + company2['name'].apply(prep) # + ' ' + company2['description'].apply(prep)

#### Ajout d'une colonne 'Version'

In [14]:
#Ajout d'une colonne de version - pour ce faire, nous créons des colonnes intermédiaires 'Win', 'Mac' et 'Linux'
company1['Win']=np.where(company1['name'].str.contains('win') | company1['description'].str.contains('win'),1,0)
company1['Mac']=np.where(company1['name'].str.contains('mac') | company1['description'].str.contains('mac'),1,0)
company1['Linux']=np.where(company1['name'].str.contains('linux') | company1['description'].str.contains('linux'),1,0)
company1['version']=np.select(
    [(company1['Win']+company1['Mac']+company1['Linux']==1) & (company1['Win']==1),
     (company1['Win']+company1['Mac']+company1['Linux']==1) & (company1['Mac']==1),
     (company1['Win']+company1['Mac']+company1['Linux']==1) & (company1['Linux']==1),
     (company1['Win']+company1['Mac']+company1['Linux']!=1)
    ],
    ['Win','Mac','Linux','None'])

company2['Win']=np.where(company2['name'].str.contains('win') | company2['description'].str.contains('win'),1,0)
company2['Mac']=np.where(company2['name'].str.contains('mac') | company2['description'].str.contains('mac'),1,0)
company2['Linux']=np.where(company2['name'].str.contains('linux') | company2['description'].str.contains('linux'),1,0)
company2['version']=np.select(
    [(company2['Win']+company2['Mac']+company2['Linux']==1) & (company2['Win']==1),
     (company2['Win']+company2['Mac']+company2['Linux']==1) & (company2['Mac']==1),
     (company2['Win']+company2['Mac']+company2['Linux']==1) & (company2['Linux']==1),
     (company2['Win']+company2['Mac']+company2['Linux']!=1)
    ],
    ['Win','Mac','Linux','None'])

### 3  - Détection des doublons

Après de nombreuses tentatives en utilisant des mesures de distance ou similarité présentes dans le package *nltk* (jaccard distance, edit distance...), nous avons testé d'autres methodes et notamment l'utilisation de l'algorithme tf-idf présent dans le package *sklearn* qui, dans sa version la plus classique, permet, pour chaque document, de calculer une mesure, par n-gram présent, proportionnelle à la fréquence du n-gram dans le document (Terme Frequency) et inversement proportionnelle à la fréquence dans la base (Inverse Document Frequency).

Une fois la fonction *TfidfVectorizer* appliquée à la concatenation d'eventuels sous-dataframes de ***company 1*** et ***company 2*** (nous y reviendrons), nous serons en mesure, pour chaque couple d'articles de calculer une mesure de similarité en effectuant un produit scalaire des deux lignes divisé par leurs normes.

Outre cette mesure de similarité, nous filtrons également nos résultats selon deux autres critères:
* si les prix retraités 'price_retreat' ne sont pas nuls, le rapport entre le plus élevé et le plus faible doit au maximum être de 2 (hypothèse que le prix d'un même produit ne peut pas aller du simple au double entre les deux bases);
* la 'Version' doit être identique ou l'une des deux doit être 'None' (ie non déterminée), un produit spécifié comme Windows ne pouvant tourner sur Mac.

Dans la pratique, la fonction *filtre_tfidf* créée ci-dessous permet, à partir des paramètres *corpus*, *ngram*, *max_df*, *sim_lim* et *stop* :
* appliquera la fonction *TfidfVectorizer* au dataframe *corpus* avec comme paramètres la taille des n-grams *ngram*, une proportion des n-grams conservés *max_df* (les plus representatifs), d'éventuels stopwords spécifiques *stop* et en activant le paramètre *sublinear_tf* (en effet, nous souhaitons privilegier la mesure idf à la mesure tf car plus appropriée à notre étude : un terme peu fréquent dans la base corpus doit être privilégié),
* calculera la similarité pour les couples où les filtres sur le prix et la version sont vérifiés;
* ajoutera les couples pour lesquels la similarité est supérieure à *sim_lim* au dataframe ***matches_df*** ;
* calculera precision, recall et f-measure actuels.

D'autres part, nous appliquerons une méthode de type *Blocking* selon l'editeur du logiciel. Comme la colonne 'manufacturer' n'est que faiblement remplie dans ***company 2***, nous selectionnerons dans la pratique les 'manufacturer' les plus importants de ***company 1*** et, un par un, ne conservons que les lignes comportant son nom (exemple: toutes les lignes comportant 'adobe', puis toutes les lignes comportant 'encore', etc...). Une fois ceci fait, nous appliquerons la fonction à toutes les lignes n'ayant pas été selectionnée comme faisant partie d'un 'match' (bien qu'il y ait des doublons dans ***company 2***, nous n'avons pas trouvé de critère discriminant permettant de les detecter).

Il nous semble important de préciser encore nos choix de paramètres :
* *corpus* comme expliqué précédemment sera la concaténation des lignes selectionnées de ***company 1*** et de ***company 2***;
* *ngram* :
    * pour les gros éditeurs, nous décidons de considérer des n-grams de 1 et 2 mots puis, sur les lignes non matchées sur des n-grams de 1 mot, via une boucle;
    * pour le reste de la base, nous décidons d'utiliser egalement une boucle, d'abord sur les  n-grams de 1 à 3 mots puis, sur les lignes non matchées sur des n-grams de 1 et 2 mots, enfin sur des n-grams de 1 mot;
* pour *max_df*, , il nous apparaît important de le limiter, particulièrement sur le reste de la base, afin de réduire les temps de calculs matriciels. Nous prenons ainsi 1% sur cette base, et 99% pour les gros éditeurs;
* le choix de la similarité minimum *sim_lim* nous apparait critique. Nous avons sélectionné 0.5 pour le reste de la base. Concernant les gros éditeurs, sachant que les corpus comprennent déjà une similarité importante (du fait de concerner un éditeur unique) nous proposons de l'augmenter et de multiplier par 1.2 la valeur seuil soit 0.6;
* enfin *stop* n'est utilisé que pour les gros éditeurs où nous supprimons le nom de l'éditeur concerné 

#### Création de la fonction *filtre_tfidf*

In [15]:
def filtre_tfidf(corpus, ngram, max_df,sim_lim, stop):
    global number_of_matches, matches, matches_df
    start = time.process_time()
    new_number_of_matches = 0
    new_matches=[]

    
    #Application de TfidfVectorizer au corpus
    vectorizer = TfidfVectorizer(ngram_range=ngram, max_df=max_df,sublinear_tf=True,stop_words=[stop]) 
    vectors = vectorizer.fit_transform(corpus['full data'])
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()

    
    for i in range(len(company1_light)):
        price1 = float(company1_light.iloc[i,6]) 
        for j in range(len(company2_light)):
            price2 = float(company2_light.iloc[j,6]) 
            #double filtre sur le ratio de prix et sur la version du logiciel
            if (price1* price2 == 0 or max(price1, price2)/min(price1, price2)<2) and (company1_light.iloc[i,11]==company2_light.iloc[j,11] or company1_light.iloc[i,11]=='None' or company2_light.iloc[j,11]=='None'):
                #calcul de la similarité
                try :
                    similarity = np.dot(dense[i],np.transpose(dense[len(company1_light)+j])).item(0)/math.sqrt(np.dot(dense[i],np.transpose(dense[i])).item(0) * np.dot(dense[len(company1_light)+j],np.transpose(dense[len(company1_light)+j])).item(0))
                except : 
                    similarity = 0
                #test de la similarité
                if  similarity > sim_lim:
                    new_number_of_matches = new_number_of_matches +1
                    new_matches.append((company1_light.iloc[i,0],company2_light.iloc[j,0]))

    #remplissage de matches_df
    if new_number_of_matches>0:
        print("New matches: {}".format(new_number_of_matches))
        number_of_matches= number_of_matches + new_number_of_matches
        print("Total matches: {}".format(number_of_matches))
        if matches== []:
            matches = new_matches 
            matches_df = pd.DataFrame(matches)
            matches_df.columns= ['idCompany1','idCompany2']
        else:
            new_matches_df = pd.DataFrame(new_matches)
            new_matches_df.columns= ['idCompany1','idCompany2']
            matches_df = pd.concat([matches_df, new_matches_df],sort=False,ignore_index=True).drop_duplicates()
        
        #calcul des mesures
        diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
        true_positives = diff_df[diff_df.Exist=='both']
        false_positives = diff_df[diff_df.Exist=='right_only']
        false_negatives = diff_df[diff_df.Exist=='left_only']
        print("Number of true positives: {}".format(len(true_positives)))
        print("Number of false positives: {}".format(len(false_positives)))
        print("Number of false negatives: {}".format(len(false_negatives)))
        precision = len(true_positives)/(len(true_positives)+ len(false_positives))
        print("Precision: {}".format(precision))
        recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
        print("Recall: {}".format(recall))
        try :
            f_measure = 2*(precision*recall)/(precision+recall)
            print("F measure: {}".format(f_measure))
        except:
            print("F measure not calculable")
    else:
        print("No new match")
    end = time.process_time()
    print("Processing time: {}".format(end - start))

In [16]:
#boucle sur les big manufacturer
big_manufacturer_list=["adobe","encore","topic","microsoft","aspyr","apple","fogware","intuit","punch","sony","nova","corel"]
number_of_matches = 0
matches=[]
matches_df=[]

for filtre in big_manufacturer_list :
    #boucle sur n-grams = (1,2) puis (1,1)
    for max_ngram in [2,1]:
        #filtre sur les éventuels matchs déjà obtenus
        try :
            company1_light=company1[~company1.id.isin(matches_df.idCompany1)]
            company2_light=company2[~company2.id.isin(matches_df.idCompany2)]    
        except : 
            company1_light=company1
            company2_light=company2        
        #filtre sur l'éditeur
        company1_light = company1_light[company1_light['full data'].str.contains(filtre)].reset_index(drop=True)
        company2_light = company2_light[company2_light['full data'].str.contains(filtre)].reset_index(drop=True)
        corpus = pd.concat([company1_light, company2_light],sort=False,ignore_index=True)
        print("{} with ngram=(1,{})".format(filtre,max_ngram))
        filtre_tfidf(corpus, (1,max_ngram), 0.99 ,0.6, filtre)
        print(" ")

adobe with ngram=(1,2)
New matches: 87
Total matches: 87
Number of true positives: 50
Number of false positives: 37
Number of false negatives: 1250
Precision: 0.5747126436781609
Recall: 0.038461538461538464
F measure: 0.07209805335255949
Processing time: 0.515625
 
adobe with ngram=(1,1)
New matches: 13
Total matches: 100
Number of true positives: 56
Number of false positives: 44
Number of false negatives: 1244
Precision: 0.56
Recall: 0.043076923076923075
F measure: 0.07999999999999999
Processing time: 0.15625
 
encore with ngram=(1,2)
New matches: 131
Total matches: 231
Number of true positives: 122
Number of false positives: 109
Number of false negatives: 1178
Precision: 0.5281385281385281
Recall: 0.09384615384615384
F measure: 0.15937295885042455
Processing time: 1.140625
 
encore with ngram=(1,1)
New matches: 18
Total matches: 249
Number of true positives: 133
Number of false positives: 116
Number of false negatives: 1167
Precision: 0.5341365461847389
Recall: 0.10230769230769231
F 

In [17]:
#Boucle sur le reste de la base avec n-grams = (1,3) puis (1,2) puis (1,1)
for max_ngram in [3,2,1]:
    #filtre sur les matchs déjà obtenus
    company1_light=company1[~company1.id.isin(matches_df.idCompany1)]
    company2_light=company2[~company2.id.isin(matches_df.idCompany2)]    
    corpus = pd.concat([company1_light, company2_light],sort=False,ignore_index=True)
    print("Other entries with ngram=(1,{})".format(max_ngram))
    filtre_tfidf(corpus, (1,max_ngram), 0.01 ,0.5, "")
    print(" ")

Other entries with ngram=(1,3)
New matches: 485
Total matches: 973
Number of true positives: 636
Number of false positives: 337
Number of false negatives: 664
Precision: 0.6536485097636177
Recall: 0.48923076923076925
F measure: 0.559612846458425
Processing time: 616.5625
 
Other entries with ngram=(1,2)
New matches: 168
Total matches: 1141
Number of true positives: 751
Number of false positives: 390
Number of false negatives: 549
Precision: 0.6581945661700263
Recall: 0.5776923076923077
F measure: 0.6153215895124949
Processing time: 290.125
 
Other entries with ngram=(1,1)
New matches: 300
Total matches: 1441
Number of true positives: 899
Number of false positives: 542
Number of false negatives: 401
Precision: 0.6238723108952117
Recall: 0.6915384615384615
F measure: 0.655964976286027
Processing time: 35.203125
 


***Au final, notre méthode nous permet, sur la base complète, d'obtenir une F-measure de 65.60% provenant d'une précision de 62.39% et d'un recall de 69.15%.***

En effet, nous avons décidé d'équilibrer relativement precision et recall, ne connaissant pas l'utilisation qui aurait été faite de notre résultat.  
Dans le cas où nous aurions dû pénaliser les faux positifs (par exemple, pour des détections d'opportunité d'arbitrage entre les deux compagnies), nous aurions pu augmenter la valeur de *sim_lim*. Au contraire, si l'objectif est de détecter un maximum des doublons (par exemple dans un objectif de ne conserver que des articles 'rares', ie les négatifs restants), nous aurions pu diminuer *sim_lim* et augmenter le ratio maximum des prix à 3.